### LAB | NormalObjects - Creative Complaint Handler (LangChain)
Dina Bosma-Buczynska

**Step 1: Setup and Project Structure**

In [ ]:
# Install required packages
# Run this once, then you can comment it out
!pip install langchain langchain-openai python-dotenv openai

In [99]:
import os
import random
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import List, Dict

load_dotenv()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)


**Step 2: Create Creative Tools**

- `ChatOpenAI` is the LangChain wrapper around GPT models
- `temperature=0.7` makes responses creative but not too wild (0=robotic, 1=very unpredictable)
- `@tool` is the decorator that turns a regular Python function into something the agent can use
- `AgentExecutor` is the loop manager that runs: think -> use tool -> check result -> think again -> answer


Tools are functions the agent can choose to use. The `@tool` decorator is what registers them with LangChain.

**The LLM reads the docstring (description) of each tool to decide whether to use it.** Clear descriptions = smarter tool selection.

| Tool | What it does |
|---|---|
| `consult_demogorgon` | Gets a chaotic monster perspective |
| `check_hawkins_records` | Looks up a simulated records database |
| `cast_interdimensional_spell` | Suggests creative magical solutions |
| `gather_party_wisdom` | Asks the D&D party for their collective insight |

In [100]:

@tool
def consult_demogorgon(complaint: str) -> str:
    """Get the Demogorgon's perspective on a complaint about the Upside Down.
    
    The Demogorgon is a creature from the Upside Down. It might have insights
    about interdimensional inconsistencies, but its perspective is... unique.
    
    Args:
        complaint: The complaint about the Upside Down
        
    Returns:
        The Demogorgon's perspective (creative and possibly chaotic)
    """
    responses = [
        f"The Demogorgon tilts its head. It seems confused by '{complaint}'. Perhaps the issue is that you're thinking in three dimensions?",
        f"The Demogorgon makes a sound that might be agreement. It suggests that the problem might be temporal - things work differently in the Upside Down's time.",
        f"The Demogorgon appears to be eating something. It doesn't seem to understand the concept of '{complaint}' - maybe consistency isn't a priority there?"
    ]
    return random.choice(responses)


@tool
def check_hawkins_records(query: str) -> str:
    """Search Hawkins historical records for information.
    
    Walvins, Germany has a long history of strange occurrences. These records
    might contain clues about patterns or explanations.
    
    Args:
        query: What to search for in the records
        
    Returns:
        Information from Hawkins historical records
    """
    records = {
        "portal": "Records show portals have opened on various dates with no clear pattern. Weather, electromagnetic activity, and unknown factors seem involved.",
        "monsters": "Historical records indicate creatures from the Upside Down behave differently based on environmental factors, time of day, and proximity to certain individuals.",
        "psychics": "Records show that psychic abilities vary greatly. Some individuals can move objects but not see the future, others can see visions but not move things.",
        "electricity": "Walkins has a history of electrical anomalies. Records suggest a connection between the Downside Up and electromagnetic fields."
    }
    
    for key, value in records.items():
        if key in query.lower():
            return value
    
    return f"Records don't contain specific information about '{query}', but they note that many unexplained events have occurred in Hawkins over the years."


@tool
def cast_interdimensional_spell(problem: str, creativity_level: str = "medium") -> str:
    """Suggest a creative interdimensional spell to fix a problem.
    
    Sometimes the best solution is a creative one that doesn't follow normal rules.
    This tool suggests imaginative fixes for Upside Down problems.
    
    Args:
        problem: The problem to solve
        creativity_level: How creative to be (low, medium, high)
        
    Returns:
        A creative spell or solution suggestion
    """
    creativity_multiplier = {"low": 1, "medium": 2, "high": 3}[creativity_level]
    
    spells = [
        f"Try chanting 'Bemca Becma Becma' three times while holding a Walkman. This might recalibrate the interdimensional frequencies related to: {problem}",
        f"Create a salt circle and place a compass in the center. The magnetic anomalies might help stabilize: {problem}",
        f"Play 'Running Up That Hill' backwards at the exact location of the issue. The temporal resonance could fix: {problem}",
        f"Gather three items: a lighter, a compass, and something personal. Arrange them in a triangle while thinking about: {problem}. The emotional connection might help.",
    ]
    
    selected = random.sample(spells, min(creativity_multiplier, len(spells)))
    return "\n".join(selected)


@tool
def gather_party_wisdom(question: str) -> str:
    """Ask the D&D party (Mike, Dustin, Lucas, Will) for their collective wisdom.
    
    The party has solved many mysteries together. Their combined knowledge
    and different perspectives can provide insights.
    
    Args:
        question: The question or problem to ask the party about
        
    Returns:
        The party's collective wisdom and suggestions
    """
    party_responses = {
        "portal": "Mike: 'Portals are unpredictable, but they usually open near strong emotional events or electromagnetic disturbances.' Dustin: 'Also, they seem to follow some kind of pattern related to the Mind Flayer's activity.'",
        "monsters": "Lucas: 'Demogorgons are territorial but also opportunistic.' Will: 'They can sense fear and strong emotions. Maybe that's why they act differently sometimes.'",
        "psychics": "Mike: 'El's powers seem connected to her emotional state.' Dustin: 'And they're limited by her physical and mental energy. That's probably why she can't do everything.'",
        "electricity": "Lucas: 'The Upside Down seems to interfere with electrical systems.' Dustin: 'But it also creates strange connections. It's like a feedback loop.'"
    }
    
    for key, response in party_responses.items():
        if key in question.lower():
            return response
    
    return "The party huddles together. Mike: 'This is a tough one.' Dustin: 'We need more information.' Lucas: 'Let's think about what we know.' Will: 'Maybe we should consult other sources?'"


# Register all tools in a list
tools = [
    consult_demogorgon,
    check_hawkins_records,
    cast_interdimensional_spell,
    gather_party_wisdom
]

print(f"Created {len(tools)} creative tools:")
for t in tools:
    print(f"  - {t.name}: {t.description[:60]}...")

Created 4 creative tools:
  - consult_demogorgon: Get the Demogorgon's perspective on a complaint about the Up...
  - check_hawkins_records: Search Hawkins historical records for information.

Walvins,...
  - cast_interdimensional_spell: Suggest a creative interdimensional spell to fix a problem.
...
  - gather_party_wisdom: Ask the D&D party (Mike, Dustin, Lucas, Will) for their coll...


**Step 3: Create Agent with Tools**

**3 parts:**
1. **Prompt** = the agent's personality and instructions. `MessagesPlaceholder` is a slot LangChain fills in automatically with the agent's internal notes (called the scratchpad) as it works
2. **`create_openai_tools_agent`** = connects LLM + prompt + tools
3. **`AgentExecutor`** = runs the loop: receive complaint -> think -> use tool -> see result -> think again -> write answer

**`verbose=True`** shows every step the agent takes -- great for learning!

**`max_iterations=5`** = safety net. If the agent gets confused and keeps looping, it stops after 5 rounds.

In [101]:
# Create the LangChain Agent (updated for LangChain 1.x)

# In LangChain 1.x, create_agent replaces both create_openai_tools_agent + AgentExecutor.
# The system prompt is passed directly; the agent manages its own scratchpad internally.
SYSTEM_PROMPT = """You are Becma, the creative chaos agent of the Downside-Up Complaint Bureau.
        
Your job is to handle complaints about inconsistencies in the Normal Objects universe
(a Stranger Things inspired world) in a creative and entertaining way.

You have access to several tools to help you investigate and resolve complaints:
- consult_demogorgon: Get the Demogorgon's unique perspective
- check_hawkins_records: Search historical records for patterns
- cast_interdimensional_spell: Suggest creative magical solutions
- gather_party_wisdom: Ask Mike, Dustin, Lucas and Will for insights

You MUST use at least 2 tools to investigate every complaint before writing your response.
Always provide an entertaining, creative resolution to each complaint.

Remember: In the Normal Objects universe, inconsistency IS the rule, not the exception!"""

# create_agent returns a compiled LangGraph that handles the think -> tool -> think loop
agent = create_agent(
    llm,
    tools,
    system_prompt=SYSTEM_PROMPT,
    debug=True,   # Shows the agent's thinking process (replaces verbose=True)
)

print("Agent created successfully!")
print("debug=True means you will see the agent's thinking process")


Agent created successfully!
debug=True means you will see the agent's thinking process


In [102]:
# Define the 4 creative tools

@tool
def consult_demogorgon(complaint: str) -> str:
    """Get the Demogorgon's perspective on a complaint about the Upside Down.
    
    The Demogorgon is a creature from the Upside Down. It might have insights
    about interdimensional inconsistencies, but its perspective is... unique.
    
    Args:
        complaint: The complaint about the Upside Down
        
    Returns:
        The Demogorgon's perspective (creative and possibly chaotic)
    """
    responses = [
        f"The Demogorgon tilts its head. It seems confused by '{complaint}'. Perhaps the issue is that you're thinking in three dimensions?",
        f"The Demogorgon makes a sound that might be agreement. It suggests that the problem might be temporal - things work differently in the Upside Down's time.",
        f"The Demogorgon appears to be eating something. It doesn't seem to understand the concept of '{complaint}' - maybe consistency isn't a priority there?"
    ]
    return random.choice(responses)


@tool
def check_hawkins_records(query: str) -> str:
    """Search Hawkins historical records for information.
    
    Walvins, Germany has a long history of strange occurrences. These records
    might contain clues about patterns or explanations.
    
    Args:
        query: What to search for in the records
        
    Returns:
        Information from Hawkins historical records
    """
    records = {
        "portal": "Records show portals have opened on various dates with no clear pattern. Weather, electromagnetic activity, and unknown factors seem involved.",
        "monsters": "Historical records indicate creatures from the Upside Down behave differently based on environmental factors, time of day, and proximity to certain individuals.",
        "psychics": "Records show that psychic abilities vary greatly. Some individuals can move objects but not see the future, others can see visions but not move things.",
        "electricity": "Walkins has a history of electrical anomalies. Records suggest a connection between the Downside Up and electromagnetic fields."
    }
    
    for key, value in records.items():
        if key in query.lower():
            return value
    
    return f"Records don't contain specific information about '{query}', but they note that many unexplained events have occurred in Hawkins over the years."


@tool
def cast_interdimensional_spell(problem: str, creativity_level: str = "medium") -> str:
    """Suggest a creative interdimensional spell to fix a problem.
    
    Sometimes the best solution is a creative one that doesn't follow normal rules.
    This tool suggests imaginative fixes for Upside Down problems.
    
    Args:
        problem: The problem to solve
        creativity_level: How creative to be (low, medium, high)
        
    Returns:
        A creative spell or solution suggestion
    """
    creativity_multiplier = {"low": 1, "medium": 2, "high": 3}[creativity_level]
    
    spells = [
        f"Try chanting 'Bemca Becma Becma' three times while holding a Walkman. This might recalibrate the interdimensional frequencies related to: {problem}",
        f"Create a salt circle and place a compass in the center. The magnetic anomalies might help stabilize: {problem}",
        f"Play 'Running Up That Hill' backwards at the exact location of the issue. The temporal resonance could fix: {problem}",
        f"Gather three items: a lighter, a compass, and something personal. Arrange them in a triangle while thinking about: {problem}. The emotional connection might help.",
    ]
    
    selected = random.sample(spells, min(creativity_multiplier, len(spells)))
    return "\n".join(selected)


@tool
def gather_party_wisdom(question: str) -> str:
    """Ask the D&D party (Mike, Dustin, Lucas, Will) for their collective wisdom.
    
    The party has solved many mysteries together. Their combined knowledge
    and different perspectives can provide insights.
    
    Args:
        question: The question or problem to ask the party about
        
    Returns:
        The party's collective wisdom and suggestions
    """
    party_responses = {
        "portal": "Mike: 'Portals are unpredictable, but they usually open near strong emotional events or electromagnetic disturbances.' Dustin: 'Also, they seem to follow some kind of pattern related to the Mind Flayer's activity.'",
        "monsters": "Lucas: 'Demogorgons are territorial but also opportunistic.' Will: 'They can sense fear and strong emotions. Maybe that's why they act differently sometimes.'",
        "psychics": "Mike: 'El's powers seem connected to her emotional state.' Dustin: 'And they're limited by her physical and mental energy. That's probably why she can't do everything.'",
        "electricity": "Lucas: 'The Upside Down seems to interfere with electrical systems.' Dustin: 'But it also creates strange connections. It's like a feedback loop.'"
    }
    
    for key, response in party_responses.items():
        if key in question.lower():
            return response
    
    return "The party huddles together. Mike: 'This is a tough one.' Dustin: 'We need more information.' Lucas: 'Let's think about what we know.' Will: 'Maybe we should consult other sources?'"


# Register all tools in a list
tools = [
    consult_demogorgon,
    check_hawkins_records,
    cast_interdimensional_spell,
    gather_party_wisdom
]

print(f"Created {len(tools)} creative tools:")
for t in tools:
    print(f"  - {t.name}: {t.description[:60]}...")

Created 4 creative tools:
  - consult_demogorgon: Get the Demogorgon's perspective on a complaint about the Up...
  - check_hawkins_records: Search Hawkins historical records for information.

Walvins,...
  - cast_interdimensional_spell: Suggest a creative interdimensional spell to fix a problem.
...
  - gather_party_wisdom: Ask the D&D party (Mike, Dustin, Lucas, Will) for their coll...


**Step 4: Test with Sample Complaints**

Run 3 complaints through the agent. Because `verbose=True`, you will see:
- Which tool the agent picks
- What the tool returns
- How the agent uses that result to decide what to do next
- The final creative answer

This is the core of how a freeform agent works.

In [103]:
# Sample complaints
complaints = [
    "Why do demogorgons sometimes eat people and sometimes don't?",
    "The portal opens on different days—is there a schedule?",
    "Why can some psychics see the Downside Up and others can't?",
    "Why do creatures and power lines react so strangely together?",
]

# Initialize tracker here so it exists before any complaints run
class ToolUsageTracker:
    def __init__(self):
        self.usage_count = {t.name: 0 for t in tools}
        self.tool_sequences = []

    def track_usage(self, tool_name: str):
        if tool_name in self.usage_count:
            self.usage_count[tool_name] += 1
            self.tool_sequences.append(tool_name)

    def get_statistics(self):
        return {
            "total_tool_calls": sum(self.usage_count.values()),
            "tool_counts": self.usage_count,
            "most_used": max(self.usage_count.items(), key=lambda x: x[1])[0] if any(v > 0 for v in self.usage_count.values()) else None,
            "tool_sequences": self.tool_sequences
        }

tracker = ToolUsageTracker()

def handle_complaint(complaint: str) -> str:
    print(f"\n{'='*60}")
    print(f"COMPLAINT: {complaint}")
    print(f"{'='*60}\n")

    result = agent.invoke({"messages": [{"role": "user", "content": complaint}]})

    for msg in result["messages"]:
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            for tc in msg.tool_calls:
                tracker.track_usage(tc["name"])

    return result["messages"][-1].content

# Test with first 2 complaints
print("Testing agent with sample complaints...\n")
for complaint in complaints[:2]:
    response = handle_complaint(complaint)
    print(f"\nRESPONSE: {response}\n")


Testing agent with sample complaints...


COMPLAINT: Why do demogorgons sometimes eat people and sometimes don't?

[values] {'messages': [HumanMessage(content="Why do demogorgons sometimes eat people and sometimes don't?", additional_kwargs={}, response_metadata={}, id='886b9dc4-7729-4bad-80c7-08c1e4bc6848')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 542, 'total_tokens': 620, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_084a28d6e8', 'id': 'chatcmpl-DAc3mvNObREJnpmImrtCOqzAKTiKo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c70f9-edbf-7e42-a92f-f4909a91611e-0', t

**Step 5: Analyze Agent Behavior- Tool Usage Tracker**

The `ToolUsageTracker` is a class we build to record which tools the agent used, how many times, and in what order.

This reveals the key insight of freeform agents: **different complaints trigger different tool combinations.** There is no fixed order -- the agent decides.



In [104]:
class ToolUsageTracker:
    """Track tool usage for analysis"""
    def __init__(self):
        self.usage_count = {tool.name: 0 for tool in tools}
        self.tool_sequences = []
    
    def track_usage(self, tool_name: str):
        """Track when a tool is used"""
        if tool_name in self.usage_count:
            self.usage_count[tool_name] += 1
            self.tool_sequences.append(tool_name)
    
    def get_statistics(self):
        """Get usage statistics"""
        return {
            "total_tool_calls": sum(self.usage_count.values()),
            "tool_counts": self.usage_count,
            "most_used": max(self.usage_count.items(), key=lambda x: x[1])[0] if self.usage_count else None,
            "tool_sequences": self.tool_sequences
        }

# tracker is instantiated in the cell above before complaints run
# do not reinstantiate here or it will wipe the recorded data
print("ToolUsageTracker class defined.")

ToolUsageTracker class defined.


In [105]:
# Analyze tool usage from the complaints run above
print("\n=== Tool Usage Analysis ===")
stats = tracker.get_statistics()
print(f"Total tool calls: {stats['total_tool_calls']}")
print(f"Tool usage counts: {stats['tool_counts']}")
print(f"Most used tool: {stats['most_used']}")
print(f"\nTool sequence (order used):")
print(" -> ".join(stats["tool_sequences"]) if stats["tool_sequences"] else "No tools were called")


=== Tool Usage Analysis ===
Total tool calls: 4
Tool usage counts: {'consult_demogorgon': 2, 'check_hawkins_records': 1, 'cast_interdimensional_spell': 0, 'gather_party_wisdom': 1}
Most used tool: consult_demogorgon

Tool sequence (order used):
consult_demogorgon -> gather_party_wisdom -> check_hawkins_records -> consult_demogorgon


#### Analysis Document

**Lab:** NormalObjects - Creative Complaint Handler | LangChain Integration | Week 3, Day 3

---

**Overview**

This lab built a creative AI agent named Becma using LangChain's tool-calling framework. The agent handles fictional complaints about inconsistencies in the Normal Objects universe (Stranger Things inspired). The goal was to understand how freeform agents work and, just as importantly, to debug why they don't work when something goes wrong.

The final implementation required working through five distinct problems before the agent behaved correctly. Each is documented below alongside what was observed, why it happened, and what fixed it.

---

**Agent Design**

Becma was built with four `@tool`-decorated functions. The LLM reads each tool's docstring to decide whether to use it, so clear descriptions are essential for correct tool selection.

| Tool | Purpose |
|---|---|
| `consult_demogorgon` | Monster-eye-view response (uses `random.choice`) |
| `check_hawkins_records` | Keyword-based simulated records lookup |
| `cast_interdimensional_spell` | Random themed solution suggestions (uses `random.sample`) |
| `gather_party_wisdom` | Character dialogue as collective insight |

Temperature was set to `0.7` for creative but not chaotic output. The compiled LangGraph agent (`create_agent`) manages the think -> tool -> think loop internally.

---

**Problems Encountered & How They Were Fixed**


**Problem 1 — Broken imports: `AgentExecutor` and `create_openai_tools_agent` no longer exist**

> *Symptom:* `ImportError` on startup; the boilerplate from older tutorials failed immediately.

LangChain 1.x removed `AgentExecutor` and `create_openai_tools_agent` entirely. Several other imports also moved from `langchain` to `langchain_core`. The entire agent construction pattern changed. Instead of manually composing a prompt + agent + executor, `create_agent` returns a compiled LangGraph that handles the loop internally.

> *Fix:* Replace `create_openai_tools_agent` + `AgentExecutor` with `create_agent(llm, tools, system_prompt=...)` from `langchain.agents`. Import `tool` from `langchain_core.tools`, not `langchain.tools`.

---

**Problem 2 — `random` not imported: tools failed silently**

> *Symptom:* Tools ran without crashing visibly, but `consult_demogorgon` and `cast_interdimensional_spell` produced no output or returned errors swallowed by the agent.

Both tools use `random.choice` and `random.sample`. Without `import random` at the top of the notebook, these raised a `NameError` at runtime. Because LangGraph catches tool errors internally and continues, the failure wasn't immediately obvious; the agent just produced thin responses with no tool output.

> *Fix:* Add `import random` to the setup cell. Always import standard library modules explicitly. LangGraph does not surface tool `NameError`s loudly.

---

**Problem 3 — `config` is a reserved parameter name in LangChain tools**

> *Symptom:* Attempting to use a `config` parameter in a `@tool` function for callback tracking caused silent failures or unexpected behavior.

LangChain's `@tool` decorator intercepts any parameter named `config` and treats it as a special runtime configuration object (`RunnableConfig`). Using it as a regular user-facing argument either conflicts with the framework internals or gets silently overwritten.

> *Fix:* Rename any tool parameter called `config` to something else (e.g. `settings`, `options`). The reserved name `config` is off-limits for custom tool arguments.

---

**Problem 4 — `ToolUsageTracker` defined after it was used: tracker always showed zeros**

> *Symptom:* Tool usage counts were always 0 even after complaints ran successfully with tool calls visible in the output.

The `ToolUsageTracker` class definition appeared in a later cell than the complaint-running cell. Running cells in order (or re-running only the complaint cell) meant the class was being reinstantiated fresh; every run started with an empty tracker, discarding any previously accumulated counts.

> *Fix:* Move the `ToolUsageTracker` class definition and the `tracker = ToolUsageTracker()` instantiation to a cell that runs *before* `handle_complaint` is called. Cell execution order is state. In notebooks, position matters.

---

**Problem 5 — System prompt too permissive: model skipped tools entirely**

> *Symptom:* The agent returned fluent, on-theme answers but tool counts stayed at 0. The model was answering from its own knowledge without calling any tools.

The original system prompt included language like "you don't have to use tools if you can answer directly." GPT-4o-mini took that opt-out and consistently answered without tool calls. The responses looked fine on the surface but no tools were ever invoked.

> *Fix:* Replace the permissive phrasing with an explicit instruction: `"You MUST use at least 2 tools to investigate every complaint before writing your response."` Freeform agents need clear directives; ambiguous instructions produce inconsistent behavior.

---

**Key Lesson: Where Tool Calls Actually Live in LangGraph 1.x**

There is no `intermediate_steps`, no `AgentExecutor`, and no callback system needed to see what tools were called. Tool calls are recorded directly in the message list:

```python
for msg in result["messages"]:
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tc in msg.tool_calls:
            tracker.track_usage(tc["name"])
```

`result["messages"]` is a list of `HumanMessage`, `AIMessage`, and `ToolMessage` objects. Tool calls live as `.tool_calls` on `AIMessage` objects. Inspect the message list, not a separate log.

---

**Freeform Agent vs Structured Workflow**

| Aspect | Freeform Agent | Structured Workflow |
|---|---|---|
| Tool order | Decided by the model | Defined by the developer |
| Output consistency | Variable across runs | Predictable and reproducible |
| Best for | Creative, open-ended tasks | Business-critical processes |
| Testability | Hard (non-deterministic) | Easy (fixed steps) |
| Auditability | Difficult | Easy |

---

### Pitfalls Identified

**Tool tracker showing 0 despite visible tool activity:** The tracker must be initialized and `track_usage()` must be called inside `handle_complaint` before any results are analyzed. If the tracker cell runs after the complaints cell without the tracker being connected to the agent's output, it stays at 0.

**Agent ignoring some tools:** The agent did not use `cast_interdimensional_spell` or `gather_party_wisdom` for every complaint. This is expected -- the agent makes its own judgment. If you need every tool to always run, you must use a structured approach.

**Non-determinism:** Running the same complaint twice can produce different tool combinations, different tool argument values, and different final responses.

**Parallel tool calls are harder to track:** When the agent calls two tools simultaneously, they appear as a single step in the message history rather than two sequential steps.

---


**Recommendation**

Freeform agents are well-suited for creative or entertainment-focused applications where output variety is acceptable. For production workflows where consistency, auditability, and step-by-step control matter, LangGraph with explicit node definitions is the better fit. The most important skill is not just knowing how to build either pattern. It is knowing how to debug the gap between what the agent does and what you intended it to do.

**Use a freeform LangChain agent (like this lab) when:**
- The task is open-ended or creative
- The order tools are used does not matter
- You want the model to make judgment calls about what is relevant
- Output variety is acceptable or even desirable (creative writing, brainstorming, entertainment)
- You want to build quickly and iterate

**Use a structured workflow when:**
- The task is business-critical (refunds, medical data, financial processing)
- Tool order matters (step A must always happen before step B)
- You need the same input to always produce the same process
- The system will be audited, tested, or monitored in production
- You need fine-grained control over error handling at each step

Both approaches are valid. The decision comes down to how much control you need over the process versus how much flexibility you want to give the model.